# django shell 기능을 jupyter notebook 이용해 작업
- django 프로젝트 root 디렉토리에서 주피터노트북을 실행해야 함
- jupyter notebook에서 django shell 사용하기 위해 환경 설정 필요

In [1]:
# django shell 환경이 아닌 곳에서 그 기능을 사용하기 위해 필요한 설정
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'   # root/config/settings.py 경로 지정
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

import django
django.setup()

In [2]:
from poll.models import Question, Choice

In [3]:
mm = Question.objects
print(type(mm))

<class 'django.db.models.manager.Manager'>


In [4]:
mm.all()

<QuerySet [<Question: 좋아하는 색깔은?>, <Question: 좋아하는 동물은?>, <Question: 가고 싶은 나라는?>, <Question: 싫어하는 동물은?>]>

모델클래스.objents.SQL메소드()[.SQL메소드()...]

# 조회(select)
- all(): 전체 조회 - 조회 결과는 QuerySet 객체 반환
    - QuerySet 객체를 통해 조회 결과를 조회하거나 추가 SQL문을 실행할 수 있다.

In [5]:
qs = Question.objects.all()   # select * from Question
print(type(qs))
print(qs.query)   # 실행된 sql문을 조회

<class 'django.db.models.query.QuerySet'>
SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question"


In [6]:
qs   # iterable

<QuerySet [<Question: 좋아하는 색깔은?>, <Question: 좋아하는 동물은?>, <Question: 가고 싶은 나라는?>, <Question: 싫어하는 동물은?>]>

In [8]:
for m in qs:
    print(type(m), m, m.question_text, m.pub_date, m.id, m.pk)   # model.pk (primary 컬럼의 값을 조회)

<class 'poll.models.Question'> 좋아하는 색깔은? 좋아하는 색깔은? 2021-10-05 08:17:30.109710+00:00 1 1
<class 'poll.models.Question'> 좋아하는 동물은? 좋아하는 동물은? 2021-10-05 08:17:52.209079+00:00 2 2
<class 'poll.models.Question'> 가고 싶은 나라는? 가고 싶은 나라는? 2021-10-05 08:17:56.685562+00:00 3 3
<class 'poll.models.Question'> 싫어하는 동물은? 싫어하는 동물은? 2021-10-05 08:18:14.431131+00:00 4 4


In [10]:
# QuerySet: iterabl - 반복문으로 조회, indexing, slicing 가능
# 단 음수 indexing은 지원하지 않는다.
qs[2]
print('id', qs[2].pk, '등록일시', qs[2].pub_date)

id 3 등록일시 2021-10-05 08:17:56.685562+00:00


In [11]:
qs[::2]

[<Question: 좋아하는 색깔은?>, <Question: 가고 싶은 나라는?>]

In [12]:
qs.first()   # 조회 결과 중 첫 번째 반환

<Question: 좋아하는 색깔은?>

In [13]:
qs.last()    # 조회 결과 중 마지막 반환

<Question: 싫어하는 동물은?>

## where절 이용한 조회
조회 조건이 있는 조회
- filter(조회 조건): 조건을 만족하는 행이 0개 이상일 경우 (해당하는 결과 없어도 에러 발생 안 됨)
- exclude(조회 조건): 조건을 만족하지 않는 0개 이상의 행들을 조회
- get(조회 조건): 조건을 만족하는 1개 행 조회. 보통 pk로 조회할 때 사용. 조회 결과가 0개 이거나 2개 이상인 경우 Exception 발생.
- filter/exclude() 반환 타입: QuerySet
- get() 반환 타입: Model 객체 반환

In [17]:
# pk(id) = 2
qs = Question.objects.filter(id=2)
type(qs)

django.db.models.query.QuerySet

In [19]:
qs.id

AttributeError: 'QuerySet' object has no attribute 'id'

In [24]:
r = qs.first()
print(r.id, r.question_text, r.pub_date)
print(type(r))

2 좋아하는 동물은? 2021-10-05 08:17:52.209079+00:00
<class 'poll.models.Question'>


In [21]:
question = Question.objects.get(pk=2)
type(question)

poll.models.Question

In [22]:
question.id, question.pk, question.question_text

(2, 2, '좋아하는 동물은?')

In [26]:
qs = Question.objects.exclude(pk=2)
print(qs.query)   # QuerySet만 .query 사용 가능
qs

SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question" WHERE NOT ("poll_question"."id" = 2)


<QuerySet [<Question: 좋아하는 색깔은?>, <Question: 가고 싶은 나라는?>, <Question: 싫어하는 동물은?>]>

In [29]:
try:
    q = Question.objects.get(pk=100)   # 조회 결과 없을 시 DoesNotExist Exception 발생
except:
    print('질문 없다')

질문 없다


In [31]:
q = Question.objects.filter(pk=100)
q

<QuerySet []>

In [32]:
q = Question.objects.get(pk__gt=1)   # where pk > 1 : 1개 이상이 조회되는 경우 MultipleObjectsReturned Exception 발생

MultipleObjectsReturned: get() returned more than one Question -- it returned 3!

In [39]:
# 조회 조건
qs = Question.objects.filter(pk__lt=3)
qs = Question.objects.filter(pk__lte=3)
qs = Question.objects.filter(pk__gt=3)
qs = Question.objects.filter(pk__gte=3)
qs = Question.objects.filter(pk__in=[1,3])   # pk in [1, 3]
qs = Question.objects.filter(pk__range=[2,4])   # pk between 2 and 4
qs = Question.objects.filter(question_text__startswith='좋아하는')   # question_text like '좋아하는%'
qs = Question.objects.filter(question_text__endswith='동물은?')   # question_text like '%동물은?'
qs = Question.objects.filter(question_text__contains='동물')   # question_text like '%동물%'
print(qs.query)
for m in qs:
    print(m.pk, m.question_text)

SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question" WHERE "poll_question"."question_text" LIKE %동물% ESCAPE '\'
2 좋아하는 동물은?
4 싫어하는 동물은?


### and로 조건을 묶는 경우
- 조회 함수들의 매개변수에 조건들을 나열해준다

In [40]:
# pk=2 and question_text like "좋아하는%"
qs = Question.objects.filter(pk=2, question_text__startswith='좋아하는')

print(qs.query)
for m in qs:
    print(m.pk, m.question_text)

SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question" WHERE ("poll_question"."id" = 2 AND "poll_question"."question_text" LIKE 좋아하는% ESCAPE '\')
2 좋아하는 동물은?


### or로 조건을 묶는 경우
- 개별 조건을 Q()에 넣고 `|` 사용

In [43]:
# pk<3 or question_text like "%동물은?"
from django.db.models import Q

# qs = Question.objects.filter(Q(pk__lt=3) | Q(question_text__endswith='동물은?'))
qs = Question.objects.filter(Q(pk__lt=3) | ~Q(question_text__endswith='동물은?'))  # not


print(qs.query)
for m in qs:
    print(m.pk, m.question_text)

SELECT "poll_question"."id", "poll_question"."question_text", "poll_question"."pub_date" FROM "poll_question" WHERE ("poll_question"."id" < 3 OR NOT ("poll_question"."question_text" LIKE %동물은? ESCAPE '\'))
1 좋아하는 색깔은?
2 좋아하는 동물은?
3 가고 싶은 나라는?


## 컬럼 선택, 정렬
- values(조회할컬럼변수나열): 컬럼 선택
    - 조회한 데이터를 dictionary에 넣어서 반환. QuerySet[{}, {}]
- order_by(기준컬럼): 정렬 기본 오름차순 / "-컬럼명" => 내림차순

In [47]:
qs = Choice.objects.all().values('choice_text', 'vote')   # select choice_text, vote from choice

print(qs.query)
qs
# for m in qs:
#     print(m.pk, m.choice_text, m.vote, m.question, sep='\t')

SELECT "poll_choice"."choice_text", "poll_choice"."vote" FROM "poll_choice"


<QuerySet [{'choice_text': '검정색', 'vote': 0}, {'choice_text': '개', 'vote': 0}, {'choice_text': '고양이', 'vote': 0}, {'choice_text': '빨강색', 'vote': 0}, {'choice_text': '프랑스', 'vote': 0}, {'choice_text': '베트남', 'vote': 0}, {'choice_text': '개', 'vote': 0}, {'choice_text': '고양이', 'vote': 0}]>

In [45]:
type(m.question)   # 모델 객체가 저장되어있음

poll.models.Question

In [48]:
for m in qs:
    print(m['choice_text'], m['vote'])    

검정색 0
개 0
고양이 0
빨강색 0
프랑스 0
베트남 0
개 0
고양이 0


In [54]:
# qs = Choice.objects.values('id', 'choice_text').all().order_by('choice_text')  # choice_text를 기준으로 asc 정렬
# qs = Choice.objects.values('id', 'choice_text').all().order_by('-choice_text')
qs = Choice.objects.values('id', 'choice_text').filter(pk__lt=3).order_by('-id', 'choice_text')

print(qs.query)
for m in qs:
    print(m['id'], m['choice_text'])

SELECT "poll_choice"."id", "poll_choice"."choice_text" FROM "poll_choice" WHERE "poll_choice"."id" < 3 ORDER BY "poll_choice"."id" DESC, "poll_choice"."choice_text" ASC
2 개
1 검정색


## 집계함수

In [57]:
# 조회한 데이터의 개수 조회
# QuerySet.count(), len(QuerySet)
qs = Choice.objects.all()
print(qs.count(), len(qs))

8 8


In [55]:
# 집계함수들 import
from django.db.models import Count, Sum, Avg, Min, Max, StdDev, Variance

- 전체 데이터셋 기준 집계
    - aggregate(집계함수(), 집계함수(), ...)
- Groupby 집계
    - values('나눌기준컬럼명', ...).annotate(집계함수(), ...)
- 결과 key: field 이름__집계이름

In [58]:
# 전체 데이터셋 기준 집계
result = Choice.objects.aggregate(Count('vote'), Sum('vote'), Avg('vote'), Min('vote'), Max('vote'), StdDev('vote'), Variance('vote'))
result

{'vote__count': 8,
 'vote__sum': 0,
 'vote__avg': 0.0,
 'vote__min': 0,
 'vote__max': 0,
 'vote__stddev': 0.0,
 'vote__variance': 0.0}

In [59]:
result['vote__avg']

0.0

In [60]:
# select min(pub_date)
# datetime, data, time 타입: min - 과거, max - 미래
result2 = Question.objects.aggregate(Min('pub_date'), Max('pub_date'))
result2

{'pub_date__min': datetime.datetime(2021, 10, 5, 8, 17, 30, 109710, tzinfo=<UTC>),
 'pub_date__max': datetime.datetime(2021, 10, 5, 8, 18, 14, 431131, tzinfo=<UTC>)}

In [61]:
min_date = result2['pub_date__min']
print(type(min_date))
min_date.year, min_date.month, min_date.day, min_date.minute, min_date.second

<class 'datetime.datetime'>


(2021, 10, 5, 17, 30)

In [62]:
# datetime.strftime('형식문자')  %Y:연도, %m:월, %d:일, %H:시간(24), %M:분, %S:초 => 원하는 형식의 문자열로 변환
r = min_date.strftime('%Y년 %m월 %d일 %H:%M:%S')
print(r, type(r))

2021년 10월 05일 08:17:30 <class 'str'>


In [63]:
# groupby
# select sum(vote), ... from choice groupby question
result3 = Choice.objects.values('question').annotate(Sum('vote'))
result3

<QuerySet [{'question': 1, 'vote__sum': 0}, {'question': 2, 'vote__sum': 0}, {'question': 3, 'vote__sum': 0}, {'question': 4, 'vote__sum': 0}]>

In [65]:
result3 = Choice.objects.values('question').annotate(Sum('vote'), Avg('vote'), Max('choice_text'))  # 문자열 min:a, max:z
for a in result3:
    print(a)

{'question': 1, 'vote__sum': 0, 'vote__avg': 0.0, 'choice_text__max': '빨강색'}
{'question': 2, 'vote__sum': 0, 'vote__avg': 0.0, 'choice_text__max': '고양이'}
{'question': 3, 'vote__sum': 0, 'vote__avg': 0.0, 'choice_text__max': '프랑스'}
{'question': 4, 'vote__sum': 0, 'vote__avg': 0.0, 'choice_text__max': '고양이'}


In [66]:
# having절 처리 함수는 없다.
# select sum(vote), ... from choice groupby question having sum(vote) > 50
for a in result3:
    if a['vote__sum'] > 50:  # having절 조건
        print(d)

## 테이블 간의 관계

### join을 이용한 조회
- 자식 테이블에서 부모 테이블 값을 조회
    - Question: 부모
    - Choice: 자식

In [68]:
c = Choice.objects.get(pk=1)
type(c), c   # Choice 모델 객체

(poll.models.Choice, <Choice: 검정색>)

In [70]:
# pk가 1인 보기의 질문?
q = c.question
type(q), q, q.question_text, q.pub_date

(poll.models.Question,
 <Question: 좋아하는 색깔은?>,
 '좋아하는 색깔은?',
 datetime.datetime(2021, 10, 5, 8, 17, 30, 109710, tzinfo=<UTC>))

In [71]:
# pk=3인 보기의 보기 내용, vote, 질문 조회
c = Choice.objects.get(pk=3)
c.choice_text, c.vote, c.question.question_text

('고양이', 0, '좋아하는 동물은?')

In [72]:
try:
    id = 3
    result = Choice.objects.get(pk=id)
    print(result.choice_text, result.vote, result.question.question_text)
except:
    print(f"{id}는 없음")

고양이 0 좋아하는 동물은?


In [74]:
# pk 2~5인 보기의 text, vote, 질문
result = Choice.objects.filter(pk__range=[2,5])
for r in result:
    print(r.pk, r.choice_text, r.vote, r.question.question_text, sep='\t')

2	개	0	좋아하는 동물은?
3	고양이	0	좋아하는 동물은?
4	빨강색	0	좋아하는 색깔은?
5	프랑스	0	가고 싶은 나라는?


- 부모 테이블에서 자식 테이블 값 조회

In [76]:
q = Question.objects.get(pk=1)
print(type(q), q)
print(q.pk, q.question_text, q.pub_date)

<class 'poll.models.Question'> 좋아하는 색깔은?
1 좋아하는 색깔은? 2021-10-05 08:17:30.109710+00:00


In [80]:
# 조회한 Question(부모)의 Choice(자식)들 조회
c_qs = q.choice_set
type(c_qs), type(Question.objects)   # 매니저 타입

(django.db.models.fields.related_descriptors.create_reverse_many_to_one_manager.<locals>.RelatedManager,
 django.db.models.manager.Manager)

In [82]:
c_qs = q.choice_set.all()
c_qs

<QuerySet [<Choice: 검정색>, <Choice: 빨강색>]>

In [83]:
for c in c_qs:
    print(c.pk, c.choice_text, c.vote, sep='\t')

1	검정색	0
4	빨강색	0


In [84]:
# pk 1~3인 질문의 질문id, 보기id, 보기 내용 조회
result = Question.objects.filter(pk__range=[1,3])
for q in result:
    print(q.pk, q.question_text)
    c_qs = q.choice_set.all()
    for c in c_qs:
        print('\t', c.pk, c.choice_text)

1 좋아하는 색깔은?
	 1 검정색
	 4 빨강색
2 좋아하는 동물은?
	 2 개
	 3 고양이
3 가고 싶은 나라는?
	 5 프랑스
	 6 베트남


# insert/update/delete
- insert/update
    - 모델객체.save()
    - 모델 객체가 insert/update할 데이터를 가지고 있다.
- delete
    - 모델객체.delete()
    - 모델객체가 삭제할 데이터의 pk를 가지고 있다.

In [85]:
def get_q_list():
    qs = Question.objects.all()
    for q in qs:
        print(q.pk, q.question_text, q.pub_date, sep='\t')
        
get_q_list()

1	좋아하는 색깔은?	2021-10-05 08:17:30.109710+00:00
2	좋아하는 동물은?	2021-10-05 08:17:52.209079+00:00
3	가고 싶은 나라는?	2021-10-05 08:17:56.685562+00:00
4	싫어하는 동물은?	2021-10-05 08:18:14.431131+00:00


In [89]:
# insert
# insert할 데이터를 가진 model 객체 생성 후 save()
from datetime import datetime
new_q = Question(question_text='좋아하는 음식은?', pub_date=datetime.now())  # pk(id)는 자동증가이므로 넣지 않음
print(new_q.pk, new_q.question_text, new_q.pub_date)
new_q.save()

None 좋아하는 음식은? 2021-10-06 16:07:31.432813


In [90]:
get_q_list()

1	좋아하는 색깔은?	2021-10-05 08:17:30.109710+00:00
2	좋아하는 동물은?	2021-10-05 08:17:52.209079+00:00
3	가고 싶은 나라는?	2021-10-05 08:17:56.685562+00:00
4	싫어하는 동물은?	2021-10-05 08:18:14.431131+00:00
5	좋아하는 음식은?	2021-10-06 07:07:31.440811+00:00


In [91]:
# update
# update할 데이터를 가진 model 객체 생성 후 save()
# pk는 테이블에 있는 값을 가지고 있어야 한다. 변경되지 않을 값은 원래 값을 그대로 가지고 있어야 한다.
update_q = Question(pk=3, question_text='가고 싶은 나라는?', pub_date=datetime.now())
# id가 3인 질문의 pub_date 값을 현재 시간으로 update
# update question set pub_date=sysdate where id=3
update_q.save()

C:\Users\Playdata\Anaconda3\envs\django\lib\site-packages\django\db\models\fields\__init__.py:1416: RuntimeWarning: DateTimeField Question.pub_date received a naive datetime (2021-10-06 16:13:14.826573) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [92]:
get_q_list()

1	좋아하는 색깔은?	2021-10-05 08:17:30.109710+00:00
2	좋아하는 동물은?	2021-10-05 08:17:52.209079+00:00
3	가고 싶은 나라는?	2021-10-06 07:13:14.826573+00:00
4	싫어하는 동물은?	2021-10-05 08:18:14.431131+00:00
5	좋아하는 음식은?	2021-10-06 07:07:31.440811+00:00


In [93]:
q = Question.objects.get(pk=3)
q.question_text = '좋아하는 꽃은?'
q.pk, q.question_text, q.pub_date

(3,
 '좋아하는 꽃은?',
 datetime.datetime(2021, 10, 6, 7, 13, 14, 826573, tzinfo=<UTC>))

In [94]:
q.save()
get_q_list()

1	좋아하는 색깔은?	2021-10-05 08:17:30.109710+00:00
2	좋아하는 동물은?	2021-10-05 08:17:52.209079+00:00
3	좋아하는 꽃은?	2021-10-06 07:13:14.826573+00:00
4	싫어하는 동물은?	2021-10-05 08:18:14.431131+00:00
5	좋아하는 음식은?	2021-10-06 07:07:31.440811+00:00


In [97]:
# delete
# 삭제할 pk값을 가진 model을 생성/ model.delete()
delete_q = Question(pk=5)
print(delete_q.pk, delete_q.question_text, delete_q.pub_date)
delete_q.delete()

5  None


(1, {'poll.Question': 1})

In [99]:
get_q_list()

1	좋아하는 색깔은?	2021-10-05 08:17:30.109710+00:00
2	좋아하는 동물은?	2021-10-05 08:17:52.209079+00:00
3	좋아하는 꽃은?	2021-10-06 07:13:14.826573+00:00
4	싫어하는 동물은?	2021-10-05 08:18:14.431131+00:00


In [101]:
delete_q = Question.objects.get(pk=2)
print(delete_q.pk, delete_q.question_text, delete_q.pub_date)

2 좋아하는 동물은? 2021-10-05 08:17:52.209079+00:00


In [102]:
delete_q.delete()
# (삭제된 총 데이터 수, {'poll.Choice': Choice 테이블에서 삭제된 데이터 수, 'poll.Question': Question 테이블에서 삭제된 데이터 수})

(3, {'poll.Choice': 2, 'poll.Question': 1})

In [103]:
get_q_list()

1	좋아하는 색깔은?	2021-10-05 08:17:30.109710+00:00
3	좋아하는 꽃은?	2021-10-06 07:13:14.826573+00:00
4	싫어하는 동물은?	2021-10-05 08:18:14.431131+00:00


In [104]:
Choice.objects.filter(question=2)

<QuerySet []>

# SQL문 직접 작성해서 실행
- 모델.objects.raw('select문')
- select문만 가능

In [107]:
sql = 'SELECT * FROM poll_choice'
rq = Choice.objects.raw(sql)   # select문 실행하고 결과를 Choice 객체에 담아준다
for d in rq:
    print(d.pk, d.choice_text, d.vote)

1 검정색 0
4 빨강색 0
5 프랑스 0
6 베트남 0
7 개 0
8 고양이 0
